## Getting started

We build a simple strategy that goes long when overnight return is negative and exits by the end of the day. If price moves against us, we use a stop loss to get out. We have 15 minute bars of Pepsi (PEP) stock prices

In [10]:
# %%checkall
import pandas as pd
import pyqstrat as pq
import numpy as np

# read 15 minute price bars
pepsi = pd.read_csv('support/pepsi_15_min_prices.csv.gz')[['date', 'c']]
pepsi.columns = ['timestamp', 'c'] 
# the date corresponding to each 15 minute timestamp
pepsi['date'] = pepsi.timestamp.values.astype('M8[D]') 
# compute overnight return
pepsi['overnight_ret'] = np.where(pepsi.date > pepsi.date.shift(1), pepsi.c / pepsi.c.shift(1) - 1, np.nan)
pepsi['overnight_ret_positive'] = (pepsi.overnight_ret > 0)  # whether overnight return is positive
# mark points just before EOD. We enter a marker order at these points so we have one bar to get filled
pepsi['eod'] = np.where(pepsi.date.shift(-2) > pepsi.date, True, False)   
# if the price drops by 0.5% after we enter in the morning take our loss and get out
pepsi['stop_price'] = np.where(np.isfinite(pepsi.overnight_ret), pepsi.c * 0.995, np.nan) 
pepsi['stop_price'] = pepsi.stop_price.fillna(method='ffill')  # fill in the stop price for the rest of the day
pepsi['stop'] = np.where(pepsi.c < pepsi.stop_price, True, False)  # whether we should exit because we are stopped out

strat_builder = pq.StrategyBuilder(data=pepsi)   
strat_builder.add_contract('PEP')
# add the stop price so we can refer to it in
strat_builder.add_indicator('stop_price', pepsi.stop_price.values) 
# convert timestamps from nanoseconds (pandas convention) to minutes so they are easier to view
timestamps = pepsi.timestamp.values.astype('M8[m]')  
prices = pepsi.c.values
# create a dictionary from contract name=>timestamp => price for use in the price function
price_dict = {'PEP': {timestamps[i]: prices[i] for i in range(len(timestamps))}}
# create the price function that the strategy will use for looking up prices 
price_function = pq.PriceFuncDict(price_dict=price_dict)
# strat_builder.set_timestamps(timestamps)  # 
strat_builder.set_price_function(price_function)

# FiniteRiskEntryRule allows us to enter trades and get out with a limited loss when a stop is hit.
# This enters market orders, if you want to use limit orders, set the limit_increment argument
entry_rule = pq.FiniteRiskEntryRule(
    reason_code='POS_OVERNIGHT_RETURN',  # this is useful to know why we entered a trade
    price_func=price_function, 
    long=True,  # whether we enter a long or short position
    percent_of_equity=0.1,  # set the position size so that if the stop is hit, we lose no more than this
    # stop price is used for position sizing.  Also, we will not enter if the price is already below 
    # stop price for long trades and vice versa
    stop_price_ind='stop_price', 
    single_entry_per_day=True)  # if we are stopped out, do we allow re-entry later in the day

# ClosePositionExitRule fully exits a position using either a market or limit order
# In this case, we want to exit at EOD so we are flat overnight
exit_rule_stop = pq.ClosePositionExitRule(   
    reason_code='STOPPED_OUT',
    price_func=price_function)

# Exit when the stop price is reached
exit_rule_eod = pq.ClosePositionExitRule(
    reason_code='EOD',
    price_func=price_function)

# Setup the rules we setup above so they are only called when the columns below in our data dataframe are true
strat_builder.add_series_rule('overnight_ret_positive', entry_rule, position_filter='zero')
strat_builder.add_series_rule('eod', exit_rule_eod, position_filter='positive')
strat_builder.add_series_rule('stop', exit_rule_stop, position_filter='positive')

# create the strategy and run it
strategy = strat_builder()
strategy.run()

[2023-10-15 18:51:34.887 __call__] Trade: 27 2019-01-15T10:00 PEP 2019-01-15 10:00:00 qty: 185563 prc: 108.12 order: PEP 2019-01-15 09:45:00 qty: 185563 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-15 18:51:34.888 __call__] Trade: 51 2019-01-15T16:00 PEP 2019-01-15 16:00:00 qty: -185563 prc: 108.98 order: PEP 2019-01-15 15:45:00 qty: -185563 EOD OrderStatus.OPEN
[2023-10-15 18:51:34.889 __call__] Trade: 79 2019-01-17T10:00 PEP 2019-01-17 10:00:00 qty: 213590 prc: 108.75 order: PEP 2019-01-17 09:45:00 qty: 213590 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-15 18:51:34.889 __call__] Trade: 103 2019-01-17T16:00 PEP 2019-01-17 16:00:00 qty: -213590 prc: 109.22 order: PEP 2019-01-17 15:45:00 qty: -213590 EOD OrderStatus.OPEN
[2023-10-15 18:51:34.890 __call__] Trade: 105 2019-01-18T10:00 PEP 2019-01-18 10:00:00 qty: 229628 prc: 109.75 order: PEP 2019-01-18 09:45:00 qty: 229628 POS_OVERNIGHT_RETURN OrderStatus.OPEN
[2023-10-15 18:51:34.890 __call__] Trade: 129 2019-01-18T16:00 PEP 2019-0

In [12]:
# Lets evaluate how the strategy did
strategy.df_roundtrip_trades()  

,symbol,multiplier,entry_timestamp,exit_timestamp,qty,entry_price,exit_price,entry_reason,exit_reason,entry_commission,exit_commission,net_pnl
0,PEP,1.0,2019-01-15 10:00:00,2019-01-15 16:00:00,185563,108.12,108.9800,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,159584.1800
1,PEP,1.0,2019-01-17 10:00:00,2019-01-17 16:00:00,213590,108.75,109.2200,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,100387.3000
2,PEP,1.0,2019-01-18 10:00:00,2019-01-18 16:00:00,229628,109.75,110.0400,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,66592.1200
3,PEP,1.0,2019-01-25 10:00:00,2019-01-25 13:15:00,240930,110.23,109.3500,POS_OVERNIGHT_RETURN,STOPPED_OUT,0.0,0.0,-212018.4000
4,PEP,1.0,2019-01-29 10:00:00,2019-01-29 11:15:00,203476,109.50,108.6979,POS_OVERNIGHT_RETURN,STOPPED_OUT,0.0,0.0,-163208.0996
5,PEP,1.0,2019-01-30 10:00:00,2019-01-30 16:00:00,173238,110.30,110.8100,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,88351.3800
6,PEP,1.0,2019-02-04 10:00:00,2019-02-04 10:45:00,184718,112.68,112.0600,POS_OVERNIGHT_RETURN,STOPPED_OUT,0.0,0.0,-114525.1600
7,PEP,1.0,2019-02-11 10:00:00,2019-02-11 10:30:00,162938,113.32,112.9300,POS_OVERNIGHT_RETURN,STOPPED_OUT,0.0,0.0,-63545.8200
8,PEP,1.0,2019-02-12 10:00:00,2019-02-12 16:00:00,152498,112.56,113.7700,POS_OVERNIGHT_RETURN,EOD,0.0,0.0,184522.5800
9,PEP,1.0,2019-02-13 10:00:00,2019-02-13 10:45:00,182540,114.19,113.7400,POS_OVERNIGHT_RETURN,STOPPED_OUT,0.0,0.0,-82143.0000


In [13]:
# Lets evaluate how the strategy did
metrics = strategy.evaluate_returns()

,gmean,amean,std,shrp,srt,k,calmar,mar,mdd_pct,mdd_dates,dd_3y_pct,dd_3y_timestamps,up_dwn,2019
,-0.04298,0.7202,0.07836,0.579,0.9765,4.719,2.055,2.055,0.3505,2019-01-24/2019-02-11,0.3505,2019-01-24/2019-02-11,7/6/0.538,-0.04298


2019-01-24T00:00 2019-02-11T00:00
